In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# Función para decodificar correos protegidos por Cloudflare
def decode_cfemail(cfemail):
    r = int(cfemail[:2], 16)
    email = ''.join([chr(int(cfemail[i:i+2], 16) ^ r) for i in range(2, len(cfemail), 2)])
    return email

# URL base con filtros seleccionados
url_base = "https://www.acotur.co/es/directorio?pagina={}&departamentos[]=Caquet%C3%A1&departamentos[]=Huila&departamentos[]=Putumayo&departamentos[]=Tolima"

# Lista para guardar los datos
data = []

# Recorremos páginas
for page in range(1, 10):  # ajusta el rango según el total de páginas
    url = url_base.format(page)
    print(f"Scrapeando página {page} -> {url}")

    response = requests.get(url)
    if response.status_code != 200:
        break

    soup = BeautifulSoup(response.text, "html.parser")
    cards = soup.find_all("div", class_="col-md-6 col-lg-4")
    if not cards:
        break

    for card in cards:
        nombre = card.find("h5", class_="card-title").get_text(strip=True) if card.find("h5") else ""
        ciudad = card.find("p", class_="card-text").get_text(strip=True) if card.find("p") else ""

        # Enlace al detalle
        link_tag = card.find("a", class_="btn-ver-mas")
        link = "https://www.acotur.co" + link_tag["href"] if link_tag else ""

        descripcion, categorias, certificaciones, email, redes, rnt = "", [], [], "", [], ""

        if link:
            detail_response = requests.get(link)
            detail_soup = BeautifulSoup(detail_response.text, "html.parser")

            # Descripción
            desc_tag = detail_soup.select_one("div.card-body p.card-text")
            descripcion = desc_tag.get_text(" ", strip=True) if desc_tag else ""

            # Categorías
            categorias = [c.get_text(strip=True) for c in detail_soup.select("div.tags-container span.badge.bg-success")]

            # Certificaciones
            certificaciones = [c.get_text(strip=True) for c in detail_soup.select("div.tags-container span.badge.bg-info")]

            # Email (Cloudflare protegido)
            email_tag = detail_soup.find("span", {"class": "__cf_email__"})
            if email_tag and email_tag.has_attr("data-cfemail"):
                email = decode_cfemail(email_tag["data-cfemail"])

            # Redes sociales
            redes = [a["href"] for a in detail_soup.select("div.social-links a[href]")]

            # RNT
            rnt_tag = detail_soup.select_one("p.badge.bg-secondary")
            if rnt_tag:
              rnt_text = rnt_tag.get_text(strip=True)
              rnt = re.sub(r"\D", "", rnt_text)  # deja solo números
            else:
              rnt = ""


        data.append({
            "Nombre": nombre,
            "Ciudad": ciudad,
            "RNT": rnt,
            "Descripción": descripcion,
            "Categorías": ", ".join(categorias),
            "Certificaciones": ", ".join(certificaciones),
            "Email": email,
            "Redes Sociales": ", ".join(redes),
            "URL": link
        })

        time.sleep(1)


df = pd.DataFrame(data)

# Separar la columna "Ciudad" en dos: Ciudad y Departamento
df[["Ciudad", "Departamento"]] = df["Ciudad"].str.split(",", n=1, expand=True)

# Quitar espacios extra al inicio o final
df["Ciudad"] = df["Ciudad"].str.strip()
df["Departamento"] = df["Departamento"].str.strip()

# Guardar en CSV
df.to_csv("acotur_filtros.csv", index=False, encoding="utf-8-sig")

print("Scraping completado. Archivo guardado como acotur_filtros.csv")


Scrapeando página 1 -> https://www.acotur.co/es/directorio?pagina=1&departamentos[]=Caquet%C3%A1&departamentos[]=Huila&departamentos[]=Putumayo&departamentos[]=Tolima
Scrapeando página 2 -> https://www.acotur.co/es/directorio?pagina=2&departamentos[]=Caquet%C3%A1&departamentos[]=Huila&departamentos[]=Putumayo&departamentos[]=Tolima
Scrapeando página 3 -> https://www.acotur.co/es/directorio?pagina=3&departamentos[]=Caquet%C3%A1&departamentos[]=Huila&departamentos[]=Putumayo&departamentos[]=Tolima
Scrapeando página 4 -> https://www.acotur.co/es/directorio?pagina=4&departamentos[]=Caquet%C3%A1&departamentos[]=Huila&departamentos[]=Putumayo&departamentos[]=Tolima
Scrapeando página 5 -> https://www.acotur.co/es/directorio?pagina=5&departamentos[]=Caquet%C3%A1&departamentos[]=Huila&departamentos[]=Putumayo&departamentos[]=Tolima
Scraping completado. Archivo guardado como acotur_filtros.csv


In [2]:
import unicodedata
# Función para normalizar texto (quita tildes y pasa a mayúsculas)
def normalizar(texto):
    if pd.isna(texto):
        return ""
    texto = texto.strip().upper()
    texto = "".join(
        c for c in unicodedata.normalize("NFD", texto)
        if unicodedata.category(c) != "Mn"
    )
    return texto
# Normalizar la columna Departamento
df["Departamento"] = df["Departamento"].apply(normalizar)

# Generar un CSV por cada departamento
for depto, datos in df.groupby("Departamento"):
    # Crear un nombre de archivo seguro (sin espacios, mayúsculas)
    nombre_archivo = f"{depto.replace(' ', '_').upper()}.csv"
    # Guardar el CSV
    datos.to_csv(nombre_archivo, index=False, encoding="utf-8-sig")
    print(f"Archivo generado: {nombre_archivo}")

Archivo generado: CAQUETA.csv
Archivo generado: HUILA.csv
Archivo generado: PUTUMAYO.csv
Archivo generado: TOLIMA.csv
